In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/widsdatathon2020/training_v2.csv
/kaggle/input/widsdatathon2020/samplesubmission.csv
/kaggle/input/widsdatathon2020/unlabeled.csv
/kaggle/input/widsdatathon2020/WiDS Datathon 2020 Dictionary.csv
/kaggle/input/widsdatathon2020/solution_template.csv


In [36]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [37]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [38]:
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)

print(train_categorical_cols)
print(len(train_categorical_cols))
print(len(train_numerical_cols))
print(test_categorical_cols)
print(len(test_categorical_cols))
print(len(test_numerical_cols))

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178
['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178


In [45]:
def replace_missing_value(df, number_features):
    imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
    df_num = df[number_features]
    imputer.fit(df_num)
    print(df[number_features].head())
    df[number_features] = imputer.transform(df_num)
    return df

In [49]:
train = replace_missing_value(train, train_numerical_cols)

TypeError: replace_missing_value() got an unexpected keyword argument 'inplace'

In [52]:
#test[test_numerical_cols].fillna(test[test_numerical_cols].mean(), inplace=True)
test[test_numerical_cols].apply(lambda x: x.fillna(x.mean()),axis=0)

TypeError: ('must be str, not int', 'occurred at index ethnicity')

In [51]:
test[test_numerical_cols].isna().any()

encounter_id                     False
patient_id                       False
hospital_id                      False
hospital_death                    True
age                               True
bmi                               True
elective_surgery                 False
height                            True
icu_id                           False
pre_icu_los_days                 False
readmission_status               False
weight                            True
albumin_apache                    True
apache_2_diagnosis                True
apache_3j_diagnosis               True
apache_post_operative            False
arf_apache                        True
bilirubin_apache                  True
bun_apache                        True
creatinine_apache                 True
fio2_apache                       True
gcs_eyes_apache                   True
gcs_motor_apache                  True
gcs_unable_apache                 True
gcs_verbal_apache                 True
glucose_apache           

In [ ]:
train.drop(train.loc[train['hospital_admit_source']=='Observation'].index, inplace=True)
#train.drop(train.loc[train['gender'].isna()].index, inplace=True)

In [ ]:
'''
for col in train_categorical_cols:
    print('\n' + col)
    print(train[col].unique())
    print(test[col].unique())
'''

In [ ]:
train.shape

In [ ]:
100 - (7907 / 91678) * 100

In [ ]:
#train.loc[train['hospital_death']== 1].count()
#91678  - 7907
83771 / 7907

In [ ]:
#train[train['apache_2_bodysystem'] == 'Undefined Diagnoses'].count()

In [ ]:
#train[train['apache_2_bodysystem'] == 'Undefined diagnoses'].count()
train.loc[train['apache_2_bodysystem'] == 'Undefined Diagnoses', 'apache_2_bodysystem'] = 'Undefined diagnoses'
test.loc[test['apache_2_bodysystem'] == 'Undefined Diagnoses', 'apache_2_bodysystem'] = 'Undefined diagnoses'

In [ ]:
train['apache_2_bodysystem'] = train['apache_2_bodysystem'].fillna("Undefined diagnoses")
test['apache_2_bodysystem'] = test['apache_2_bodysystem'].fillna("Undefined diagnoses")

In [ ]:
train['ethnicity'] = train['ethnicity'].fillna("Other/Unknown")
test['ethnicity'] = test['ethnicity'].fillna("Other/Unknown")

In [ ]:
train['hospital_admit_source'] = train['hospital_admit_source'].fillna("Other")
test['hospital_admit_source'] = test['hospital_admit_source'].fillna("Other")

In [ ]:
train['icu_admit_source'] = train['icu_admit_source'].fillna("Other ICU")
test['icu_admit_source'] = test['icu_admit_source'].fillna("Other ICU")

In [ ]:
train['gender'] = train['gender'].fillna("NA")
test['gender'] = test['gender'].fillna("NA")

In [ ]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("NA")
test[test_categorical_cols] = test[test_categorical_cols].fillna("NA")

In [ ]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()
allcols = train.columns.values.tolist()
inputcols = allcols[4:]
outputcols = allcols[3]

In [ ]:
X = train[inputcols]
y = train[outputcols]

In [ ]:
train_pool = Pool(data=X,label = y,cat_features=train_categorical_cols,
#                   baseline= X_train[""], ## 
#                   group_id = X_train['hospital_id']
                 )

In [ ]:

predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.04,depth = 10,loss_function='Logloss',l2_leaf_reg = 4,iterations=600, random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])


In [ ]:
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("2701.csv",index=False)

In [ ]:
print('completed')